frame Extraction and face detection

In [2]:
import cv2
import os

# Load video file
cap = cv2.VideoCapture("D:\\utube\\utube\\student19 ancy.mp4")

# Define video frame rate and number of frames per minute
fps = cap.get(cv2.CAP_PROP_FPS)
frames_per_minute = int(fps * 2.4)

# Initialize minute counter and frame counter
minute_count = 0
frame_count = 0

# Path to the output folder for the extracted frames
frames_folder = "D:\\learning_frames_faces_eyes\\own_dataset_frames\\ANCY\\"
os.makedirs(frames_folder, exist_ok=True)

# Create the output folders for faces and eyes
output_folder_eyes = "D:\\learning_frames_faces_eyes\\survey_eyes\\survey_eyes\\ANCY\\"
output_folder_faces =  "D:\\learning_frames_faces_eyes\\survey_faces\\survey_faces\\ANCY\\"
os.makedirs(output_folder_faces, exist_ok=True)
os.makedirs(output_folder_eyes, exist_ok=True)

# Load the face and eye detection classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Loop through video frames
while True:
    # Read next frame
    ret, frame = cap.read()
    if not ret:
        break

    # Increment frame counter
    frame_count += 1

    # Calculate current minute
    current_minute = int(frame_count / frames_per_minute)

    # If current minute is different from previous minute, process frame
    if current_minute != minute_count:
        # Extract current frame and save as image file
        cv2.imwrite(f'D:\\learning_frames_faces_eyes\\own_dataset_frames\\ANCY\\frame_{current_minute}.jpg', frame)
        minute_count = current_minute

        # Load the image
        img = frame
        
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        # Loop through all the detected faces
        for (x,y,w,h) in faces:
            # Extract the face from the image
            face = img[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (224, 224), interpolation=cv2.INTER_LANCZOS4)
            
            # Save the extracted face to the output folder without bounding box
            cv2.imwrite(os.path.join(output_folder_faces, f"{frame_count}.jpg"), face_resized)
            
            # Detect eyes in the face region
            eyes = eye_cascade.detectMultiScale(face_resized)
            if len(eyes) >= 2:
                    x1, y1, w1, h1 = eyes[0]
                    x2, y2, w2, h2 = eyes[1]
                    eye_pair = face_resized[min(y1, y2):max(y1+h1, y2+h2), min(x1, x2):max(x1+w1, x2+w2)]
                    eye_pair_resized = cv2.resize(eye_pair, (64, 64), interpolation=cv2.INTER_LINEAR)
                    # Save the extracted eye to the output folder with bounding box
                    cv2.imwrite(os.path.join(output_folder_eyes, f"{frame_count}.jpg"), eye_pair_resized)


FOCUS DETECTION CNN ARCHITECTURE

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths for the training and testing data
train_path = "D:\\dataset\\dataset\\train"
test_path = "D:\\dataset\\dataset\\test"
val_path="D:\\dataset\\dataset\\validate"
# Define the CNN architecture
input_layer = Input(shape=(64, 64, 3))
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
conv2 = Conv2D(64, (3, 3), activation='relu')(conv1)
pooling = MaxPooling2D(pool_size=(2, 2))(conv2)
flatten = Flatten()(pooling)
fc = Dense(2, activation='softmax')(flatten)

model = Model(inputs=input_layer, outputs=fc)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 64)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dense (Dense)               (None, 2)                 115202    
                                                             

In [2]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)


In [3]:
# Set the parameters for the data generator
datagen = ImageDataGenerator(rescale=1./255)

# Generate the training data from the training folder
train_generator = datagen.flow_from_directory(
        train_path,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Generate the testing data from the testing folder
test_generator = datagen.flow_from_directory(
        test_path,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')
val_generator = datagen.flow_from_directory(
        val_path,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 14400 images belonging to 2 classes.
Found 4800 images belonging to 2 classes.
Found 4800 images belonging to 2 classes.


In [4]:
from keras.models import load_model
model = load_model("C:\\Users\\DELL\\Downloads\\code\\f1.h5")

In [5]:
# Evaluate the model on the testing data
score = model.evaluate(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

150/150 [==============================] - 133s 891ms/step - loss: 0.9193 - accuracy: 0.7027
Test loss: 0.9193004369735718
Test accuracy: 0.7027083039283752


draw bounding box for faces and eyes

In [19]:
import cv2
import os

def detect_face(predicted_class):
    # Load the face and eye cascade classifiers
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    folder_path = "D:\\learning_frames_faces_eyes\\survey_faces\\survey_faces\\ANCY\\"

    # Initialize lists to store the coordinates of the bounding boxes
    face_positions = []
    eye_positions = []

    # Loop through each frame in the folder
    for i, filename in enumerate(sorted(os.listdir(folder_path))):
        if filename.endswith('.jpg'):
            # Read the image
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)

            # Resize the image to 500x500
            img = cv2.resize(img, (500, 500))



            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Detect faces in the grayscale image
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

            # Draw bounding boxes around the faces and eyes
            for (x, y, w, h) in faces:
                cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
                face_positions.append((i, x, y, w, h)) # Add face position to list
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = img[y:y+h, x:x+w]
                eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)
                for (ex, ey, ew, eh) in eyes:
                    cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
                    eye_positions.append((i, x+ex, y+ey, ew, eh)) # Add eye position to list

            # Add predicted class label to image
            if predicted_class == 0:
                cv2.rectangle(img, (10, 10), (200, 50), (255, 255, 255), -1)
                cv2.putText(img, " FOCUSED", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            else:
                cv2.rectangle(img, (10, 10), (200, 50), (255, 255, 255), -1)
                cv2.putText(img, f" NOT FOCUSED", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Display the image
            cv2.imshow('Face and eyes detection', img)
            cv2.waitKey(1)

    # Release the video capture object and close all windows
    cv2.destroyAllWindows()


In [25]:
from tensorflow.keras.preprocessing import image
import numpy as np
from keras.models import load_model
import tensorflow as tf
import os


# Load the trained model
model = load_model("C:\\Users\\DELL\\Downloads\\code\\f1.h5")

# Path to the folder containing the images to be processed
folder_path = "D:\\learning_frames_faces_eyes\\survey_eyes\\survey_eyes\\ANCY\\"
class_names = ['focused', 'not-focused']
# Loop over each image file in the folder
for dirpath, dirnames, filenames in os.walk(folder_path):
    for filename in filenames:
        if filename.endswith('.jpg'):
            # Load and preprocess the image
            img_path = os.path.join(dirpath, filename)
            img = image.load_img(img_path, target_size=(64,64))
            img = np.expand_dims(img, axis=0)

            # Make a prediction using the loaded model
            result = model.predict(img)
            y_pred = np.argmax(result[0])
            print(f"{filename}:{result} ,{filename}: {y_pred}")
predicted_class = np.argmax(result)

print(f'Predicted class: {predicted_class}')

# Print the predicted class
if predicted_class == 0:
    print("focused")
    detect_face(0)
else:
    print("not-focused")
    detect_face(1)

1/1 [==============================] - 0s 60ms/step
105.jpg:[[1.2356497e-31 1.0000000e+00]] ,105.jpg: 1
1/1 [==============================] - 0s 28ms/step
1050.jpg:[[1. 0.]] ,1050.jpg: 0
1/1 [==============================] - 0s 22ms/step
1085.jpg:[[1. 0.]] ,1085.jpg: 0
1/1 [==============================] - 0s 22ms/step
1120.jpg:[[1. 0.]] ,1120.jpg: 0
1/1 [==============================] - 0s 27ms/step
1155.jpg:[[1. 0.]] ,1155.jpg: 0
1/1 [==============================] - 0s 26ms/step
1190.jpg:[[1. 0.]] ,1190.jpg: 0
1/1 [==============================] - 0s 21ms/step
1225.jpg:[[1.0000000e+00 1.1830155e-25]] ,1225.jpg: 0
1/1 [==============================] - 0s 22ms/step
1260.jpg:[[1. 0.]] ,1260.jpg: 0
1/1 [==============================] - 0s 31ms/step
1295.jpg:[[1. 0.]] ,1295.jpg: 0
1/1 [==============================] - 0s 25ms/step
1365.jpg:[[1. 0.]] ,1365.jpg: 0
1/1 [==============================] - 0s 26ms/step
140.jpg:[[1. 0.]] ,140.jpg: 0
1/1 [==============================]

455.jpg:[[1. 0.]] ,455.jpg: 0
1/1 [==============================] - 0s 29ms/step
4550.jpg:[[1. 0.]] ,4550.jpg: 0
1/1 [==============================] - 0s 21ms/step
4585.jpg:[[1. 0.]] ,4585.jpg: 0
1/1 [==============================] - 0s 26ms/step
4620.jpg:[[1. 0.]] ,4620.jpg: 0
1/1 [==============================] - 0s 24ms/step
525.jpg:[[1. 0.]] ,525.jpg: 0
1/1 [==============================] - 0s 23ms/step
560.jpg:[[1. 0.]] ,560.jpg: 0
1/1 [==============================] - 0s 24ms/step
595.jpg:[[1. 0.]] ,595.jpg: 0
1/1 [==============================] - 0s 24ms/step
665.jpg:[[1. 0.]] ,665.jpg: 0
1/1 [==============================] - 0s 22ms/step
70.jpg:[[1. 0.]] ,70.jpg: 0
1/1 [==============================] - 0s 27ms/step
700.jpg:[[1. 0.]] ,700.jpg: 0
1/1 [==============================] - 0s 25ms/step
735.jpg:[[1. 0.]] ,735.jpg: 0
1/1 [==============================] - 0s 28ms/step
805.jpg:[[1. 0.]] ,805.jpg: 0
1/1 [==============================] - 0s 26ms/step
840.jpg:[[1.

emotion detection for emotion wise grouping

In [9]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
import os

model = load_model("C:\\Users\\DELL\\Downloads\\code\\new.h5")
main_folder_path = "D:\\learning_frames_faces_eyes\\survey_faces\\survey_faces\\"
emotions = ['anger', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

for folder_name in os.listdir(main_folder_path):
    folder_path = os.path.join(main_folder_path, folder_name)
  
    if not os.path.isdir(folder_path):
        continue
  
    emotion_counts = [0] * len(emotions)
  
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):
            img_path = os.path.join(folder_path, filename)
            img = image.load_img(img_path, target_size=(48,48))
            img = np.expand_dims(img, axis=0)
            # Make a prediction using the loaded model
            result = model.predict(img)
            y_pred = np.argmax(result[0])
          
            # Count the predicted emotion
            emotion_counts[y_pred] += 1

    # Get the mode emotion
    mode_class = np.argmax(emotion_counts)
    print(mode_class)
    label = emotions[mode_class]
    print(f"{folder_name}: {label}")
    

1/1 [==============================] - 0s 26ms/step
5
ANCY: sad
1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 30ms/step
5
ancy_utube: sad
0
nithesh: anger
1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 33ms/step
5
sandhya_utube: sad
1/1 [==============================] - 0s 31ms/step
3
student10_faces: happy
1/1 [==============================] - 0s 31ms/step
3
student11_faces: happy
1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 25ms/step
2
student12_faces: fear
1/1 [==============================] - 0s 30ms/step
3
student13_faces: happy
1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 34ms/step
3
student14_faces: happy
1/1 [==============================] - 0s 29ms/step
5
student15_faces: sad
1/1 [==============================] - 0s 25ms/step


...

1/1 [==============================] - 0s 28ms/step
5
student3_faces: sad
1/1 [==============================] - 0s 26ms/step
5
student4_faces: sad
1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 27ms/step
3
student5_faces: happy
1/1 [==============================] - 0s 39ms/step
0
student6_faces: anger
1/1 [==============================] - 0s 30ms/step
3
student7_faces: happy
1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 29ms/step
4
student8_faces: neutral
1/1 [==============================] - 0s 34ms/step
5
student9_faces: sad
1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 33ms/step
4
tania_utube: neutral
1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 28ms/step
3
venki: happy


In [ ]:
# EMOTION CLASS PREDICTION AND DEP CALCULATION

In [17]:
import cv2
import os

def split_video_into_segments(video_file_path, output_dir_path, segment_length=40, frames_per_segment=35):
    # Open the video file
    video_file = cv2.VideoCapture(video_file_path)

    # Get the frame rate of the video
    fps = int(video_file.get(cv2.CAP_PROP_FPS))

    # Calculate the number of frames in each segment
    segment_frames = fps * segment_length

    # Initialize the frame counter
    frame_count = 0

    # Initialize the segment counter
    segment_count = 1

    # Create a directory for the output frames
    if not os.path.exists(output_dir_path):
        os.mkdir(output_dir_path)

    # Loop through the video
    while True:
        # Read a frame from the video
        ret, frame = video_file.read()

        # Check if the frame was successfully read
        if not ret:
            break

        # Increment the frame counter
        frame_count += 1

        # Check if the current frame is the last frame of the current segment
        if frame_count % segment_frames == 0:
            # Save the segment as a new video file
            segment_name = f'segment_{segment_count}.mp4'
            segment_file = cv2.VideoWriter(os.path.join(output_dir_path, segment_name), cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame.shape[1], frame.shape[0]))

            # Create a directory for the output frames for this segment
            segment_dir = os.path.join(output_dir_path, f"question_{segment_count}")
            if not os.path.exists(segment_dir):
                os.mkdir(segment_dir)

            # Initialize the frame counter for this segment
            segment_frame_count = 0

            # Loop through the frames in the segment
            for i in range(frames_per_segment):
                # Calculate the index of the frame to read
                frame_index = int(i * (segment_frames / frames_per_segment))

                # Set the video file position to the index of the frame to read
                video_file.set(cv2.CAP_PROP_POS_FRAMES, frame_count - segment_frames + frame_index)

                # Read the frame
                ret, frame = video_file.read()

                # Check if the frame was successfully read
                if not ret:
                    break

                # Save the frame as an image file
                frame_name = os.path.join(segment_dir, f'frame_{segment_frame_count}.jpg')
                cv2.imwrite(frame_name, frame)
                segment_frame_count += 1

                # Write the frame to the segment video file
                segment_file.write(frame)

            # Release the segment video file
            segment_file.release()

            # Increment the segment counter
            segment_count += 1

    # Release the video file
    video_file.release()

In [10]:
import cv2
import os

# Load the Haar Cascade Classifier files for face and eyes
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

def detect_faces_and_eyes(input_folder_path, output_face_folder_path, output_eye_folder_path):
    # Create directories for the output face and eye images
    if not os.path.exists(output_face_folder_path):
        os.mkdir(output_face_folder_path)

    if not os.path.exists(output_eye_folder_path):
        os.mkdir(output_eye_folder_path)

    # Loop through each frame folder and extract faces and eyes
    num_faces_list = []
    num_eyes_list = []
    for folder_name in os.listdir(input_folder_path):
        if folder_name.startswith('question'):
            folder_path = os.path.join(input_folder_path, folder_name)
            face_folder_path = os.path.join(output_face_folder_path, folder_name)
            eye_folder_path = os.path.join(output_eye_folder_path, folder_name)

            if not os.path.exists(face_folder_path):
                os.mkdir(face_folder_path)

            if not os.path.exists(eye_folder_path):
                os.mkdir(eye_folder_path)

            # Loop through each frame in the folder and extract faces and eyes
            num_faces = 0
            num_eyes = 0
            for frame_name in os.listdir(folder_path):
                frame_path = os.path.join(folder_path, frame_name)
                frame = cv2.imread(frame_path)

                # Convert the frame to grayscale for face detection
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detect faces in the frame
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

                # Loop through each face and extract eyes
                for (x, y, w, h) in faces:
                    face = frame[y:y+h, x:x+w]
                    face_name = os.path.join(face_folder_path, frame_name.replace('.jpg', f'_face{x}_{y}.jpg'))
                    cv2.imwrite(face_name, face)

                    # Convert the face to grayscale for eye detection
                    gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

                    # Detect eyes in the face
                    eyes = eye_cascade.detectMultiScale(gray_face, scaleFactor=1.3, minNeighbors=5)

                    # Extract eye pairs
                    if len(eyes) >= 2:
                        x1, y1, w1, h1 = eyes[0]
                        x2, y2, w2, h2 = eyes[1]
                        eye_pair = face[min(y1, y2):max(y1+h1, y2+h2), min(x1, x2):max(x1+w1, x2+w2)]
                        eye_path = os.path.join(eye_folder_path, frame_name.replace('.jpg', f'_eye{x}_{y}.jpg'))
                        cv2.imwrite(eye_path, eye_pair)
                        num_eyes += 1

                num_faces += len(faces)

            num_faces_list.append(num_faces)
            num_eyes_list.append(num_eyes)

    return num_faces_list, num_eyes_list


In [21]:
input_video_path ="D:\\quiz_vdo\\ancy.mp4"
output_folder ="D:\\quiz_frames_faces_eyes\\extracted_frames\\ancy\\"
#segment_duration = 10  # seconds
split_video_into_segments(input_video_path, output_folder)


In [ ]:
input_folder_path = "D:\\quiz_frames_faces_eyes\\extracted_frames\\ancy\\"
output_face_folder_path = "D:\\quiz_faces\\\fancy\\"
output_eye_folder_path = "D:\\quiz_eyes\\fancy"
num_faces_list, num_eyes_list=detect_faces_and_eyes(input_folder_path, output_face_folder_path, output_eye_folder_path)

In [1]:
import cv2
import os

def detect_faces(mode_emotion):
    folder_path ="D:\\quiz_faces\\aadrin"

    # Loop through each subfolder and its files
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith('.jpg'):
                # Read the image
                img_path = os.path.join(dirpath, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (600, 600))


                # Add predicted class label to image
                if mode_emotion == 'angry':
                    cv2.rectangle(img, (10, 10), (200, 50), (255, 255, 255), -1)
                    cv2.putText(img, " ANGRY", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif mode_emotion == 'disgust':
                    cv2.rectangle(img, (10, 10), (200, 50), (255, 255, 255), -1)
                    cv2.putText(img, " DISGUST", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif mode_emotion == 'fear':
                    cv2.rectangle(img, (10, 10), (200, 50), (255, 255, 255), -1)
                    cv2.putText(img, " FEAR", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif mode_emotion=='happy':
                    cv2.rectangle(img, (10, 10), (200, 50), (255, 255, 255), -1)
                    cv2.putText(img, " HAPPY", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif mode_emotion == 'neutral':
                    cv2.rectangle(img, (10, 10), (200, 50), (255, 255, 255), -1)
                    cv2.putText(img, " NEUTRAL", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif mode_emotion == 'sad':
                    cv2.rectangle(img, (10, 10), (200, 50), (255, 255, 255), -1)
                    cv2.putText(img, " SAD", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.rectangle(img, (10, 10), (200, 50), (255, 255, 255), -1)
                    cv2.putText(img, f" SURPRISE", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                # Display the image
                cv2.imshow('Emotion detection', img)
                cv2.waitKey(1)

    # Release the video capture object and close all windows
    cv2.destroyAllWindows()


In [2]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
import os
from statistics import mode
import cv2

# Load the pre-trained model
model = load_model("C:\\Users\\DELL\\Downloads\\code\\new.h5")

# Define the directory path containing the face images
folder_path = "D:\\quiz_faces\\fmadhu\\"

# Define the emotions to predict
emotions = ["anger", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
# Initialize a list to store the mode of y_preds list for each subfolder
mode_emotion_indices = []
mode_emotion_probabilities=[]

# Loop through each subfolder in the directory
for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        
        # Initialize a dictionary to store the probabilities for each emotion
        emotion_probabilities = {emotion: [] for emotion in emotions}

        # Initialize a list to store the predicted emotions for each image
        y_preds = []
        
        # Loop through each image file in the subfolder
        for filename in os.listdir(subfolder_path):
            if filename.endswith('.jpg'):
                img_path = os.path.join(subfolder_path, filename)
                
                # Load the image and preprocess it for prediction
                img = image.load_img(img_path, target_size=(48,48))
                img = np.expand_dims(img, axis=0)
                
                # Predict the probabilities for each emotion
                result = model.predict(img)
                y_pred = np.argmax(result[0])
                y_preds.append(y_pred)
                for i, emotion in enumerate(emotions):
                    emotion_probabilities[emotion].append(result[0][i])
        
        # Find the mode emotion for the subfolder
        mode_emotions = [emotions[i] for i in y_preds]
        mode_emotion = mode(mode_emotions)
        mode_emotion_probability = max(emotion_probabilities[mode_emotion])
        mode_emotion_probabilities.append(mode_emotion_probability)
        
        # Get the index of the mode emotion in the emotions list and append it to the mode_emotion_indices list
        mode_emotion_index = emotions.index(mode_emotion)
        mode_emotion_indices.append(mode_emotion_index)
        
        print(f"{subfolder}: Mode emotion: {mode_emotion}, Mode emotion probability: {mode_emotion_probability}")
       
        if mode_emotion=='angry':
            detect_faces('angry')
        elif mode_emotion=='disgust':
            detect_faces('disgust')
        elif mode_emotion=='fear':
            detect_faces('fear')
        elif mode_emotion=='happy':
            detect_faces('happy')
            
        elif mode_emotion=='neutral':
            detect_faces('neutral')
        elif mode_emotion=='sad':
            detect_faces('sad')
        else:
            detect_faces('surprise')
        
# Print the list of mode emotion probabilities and mode emotion indices for each subfolder
print("Emotion_probabilities",mode_emotion_probabilities)
print("Mode_emotion_indicies",mode_emotion_indices)


1/1 [==============================] - 0s 53ms/step
question_1: Mode emotion: happy, Mode emotion probability: 0.9993392825126648
1/1 [==============================] - 0s 43ms/step
question_10: Mode emotion: happy, Mode emotion probability: 0.9925683736801147
1/1 [==============================] - 0s 43ms/step
question_2: Mode emotion: happy, Mode emotion probability: 0.987433135509491
1/1 [==============================] - 0s 50ms/step
question_3: Mode emotion: happy, Mode emotion probability: 0.9997292160987854
1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 30ms/step
question_4: Mode emotion: happy, Mode emotion probability: 0.999514102935791
1/1 [==============================] - 0s 29ms/step
question_5: Mode emotion: happy, Mode emotion probability: 0.9994149208068848
1/1 [==============================] - 0s 30ms/step
question_6: Mode emotion: happy, Mode emotion probability: 0.9975904226303101
1/1 [==============================] - 0s 29ms/step
question_7: Mode emotion: happy, Mode emotion probability: 0.9954073429107666
1/1 [==============================] - 0s 26ms/step
question_8: Mode emotion: happy, Mode emotion probability: 0.9886658191680908
1/1 [==============================] - 0s 27ms/step
question_9: Mode emotion: happy, Mode emotion probability: 0.9971548318862915
Emotion_probabilities [0.9993393, 0.9925684, 0.98743314, 0.9997292, 0.9995141, 0.9994149, 0.9975904, 0.99540734, 0.9886658, 0.99715483]
Mode_emotion_indicies [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [3]:
import cv2
import os

def classification(avg_CI):
    # Load the face and eye cascade classifiers
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    folder_path ="D:\\learning_frames_faces_eyes\\survey_faces\\survey_faces\\student1_faces\\"

    # Initialize lists to store the coordinates of the bounding boxes
    face_positions = []
    eye_positions = []

    # Loop through each frame in the folder
    for i, filename in enumerate(sorted(os.listdir(folder_path))):
        if filename.endswith('.jpg'):
            # Read the image
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)

            # Resize the image to 500x500
            img = cv2.resize(img, (500, 500))



            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Detect faces in the grayscale image
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

            # Draw bounding boxes around the faces and eyes
            for (x, y, w, h) in faces:
                cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
                face_positions.append((i, x, y, w, h)) # Add face position to list
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = img[y:y+h, x:x+w]
                eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)
                for (ex, ey, ew, eh) in eyes:
                    cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
                    eye_positions.append((i, x+ex, y+ey, ew, eh)) # Add eye position to list

            # Add predicted class label to image
            if avg_CI>=50:
               
                cv2.putText(img, " HIGHLY ENGAGED", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
            elif avg_CI>0 and avg_CI<50:
                
                cv2.putText(img, " NOMINALLY ENGAGED", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            else:
                cv2.rectangle(img, (10, 10), (200, 50), (255, 255, 255), -1)
                cv2.putText(img, f" NOT ENGAGED", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Display the image
            cv2.imshow('ENGAGEMENT DETECTION', img)
            cv2.waitKey(1)

    # Release the video capture object and close all windows
    cv2.destroyAllWindows()


In [9]:
CI=[]
for i in range(len(mode_emotion_probabilities)):
    if mode_emotion_indices[i] == 0:
        ci = (mode_emotion_probabilities[i] * 0.5) * 100
        CI.append(ci)
    elif mode_emotion_indices[i] == 1:
        ci = ( mode_emotion_probabilities[i] * 0) * 100
        CI.append(ci)
    elif mode_emotion_indices[i] == 2:
        ci = ( mode_emotion_probabilities[i] * 0.12) * 100
        CI.append(ci)
    elif mode_emotion_indices[i]== 3:
        ci =  (mode_emotion_probabilities[i] * 0.68) * 100
        CI.append(ci)
    elif mode_emotion_indices[i]== 4:
        ci = ( mode_emotion_probabilities[i] * 0.75) * 100
        CI.append(ci)
    elif mode_emotion_indices[i]== 5:
        ci = ( mode_emotion_probabilities[i] * 0.628) * 100
        CI.append(ci)
    else:
        ci = ( mode_emotion_probabilities[i] * 0.628) * 100
        CI.append(ci)
   
    print(ci)
print(CI)
avg_CI = sum(CI) / len(CI)
print("OVERALL CI",avg_CI)


67.9550712108612
67.4946494102478
67.14545321464539
67.98158669471741
67.9669589996338
67.96021461486816
67.83614873886108
67.68769931793214
67.22927570343018
67.80652856826784
[67.9550712108612, 67.4946494102478, 67.14545321464539, 67.98158669471741, 67.9669589996338, 67.96021461486816, 67.83614873886108, 67.68769931793214, 67.22927570343018, 67.80652856826784]
OVERALL CI 67.70635864734649


In [10]:
if avg_CI>50:
    print("HIGHLY ENGAGED...!!!")
    classification(avg_CI)
    
elif avg_CI>1 and avg_CI<=50:
    classification(avg_CI)
    print("NOMINALLY ENGAGED...!!!")
else:
    classification(avg_CI)
    PRINT("NOT ENGAGED...!!!")


HIGHLY ENGAGED...!!!
